# Agentic AI with LLMs in a Loop
This Google Colab notebook demonstrates a simple Agentic AI framework using OpenAI's API.

We'll build three agents:
- **Writing Agent**: Generates a marketing plan based on a product description
- **Critiquing Agent**: Provides feedback on the marketing plan
- **Decision Agent**: Determines if the plan is good enough to finalize

Finally, an **Orchestrator** will run these agents in a loop to iteratively improve the output.

Let's get started!

In [ ]:
# 📦 Prerequisites
!pip install openai --quiet


In [ ]:
# Imports
from openai import OpenAI
import os
from google.colab import userdata


In [ ]:
# 🔑 Set your OpenAI API key
client = OpenAI(api_key=userdata.get('openai_api_key'))

In [ ]:
# 📝 Sample Product Description
product_description = """
Introducing AquaGlow: a smart, eco-friendly water bottle that reminds users to stay hydrated
with LED light cues and syncs with mobile health apps to track water intake.
"""

In [ ]:
# ✍️ Writing Agent
def writing_agent(description, critique=None, previous_plan=None):
    if critique and previous_plan:
        prompt = f"""
        Here's a product description:{description}

        Here is the previous marketing plan:{previous_plan}

        Here is the critique of that plan:{critique}

        Please write a full-blown marketing plan based on the description, previous marketing plan, and the critique.
        """
    else:
        prompt = f"""
        Here's a product description:{description}

        Please write a first-draft high-level marketing plan.
        """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

In [ ]:
# 🧐 Supportive Critiquing Agent (Balanced & Action-Oriented)
def critiquing_agent(description: str, plan: str) -> str:
    prompt = f"""
      You are an experienced marketing strategist providing constructive feedback on a product's marketing plan.

      Product Description:
      {description}

      Marketing Plan:
      {plan}

      Your critique should include:

      1. **Strengths** – Identify what works well and explain why.
      2. **Key Areas for Improvement** – Point out significant gaps or vague areas of improvement. If there are none, just say "None."
      3. **Risks or Missed Opportunities** – Flag potential risks or strategic blind spots. If there are none, just say "None."
      4. **Recommendations** – Provide clear, actionable suggestions for strengthening the plan. If there are none, just say "None."
      5. **Tone** – Professional, helpful, and forward-looking. Be honest but encouraging—you're helping a capable peer improve their work.

      Your goal is not perfection, but meaningful refinement. Be insightful and supportive.
      """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a constructive, insightful marketing strategist and reviewer."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=800
    )
    return response.choices[0].message.content.strip()


In [ ]:
# ✅ Balanced Decision Agent with Critique-Aware Evaluation
def decision_agent(description: str, plan: str, critique: str) -> str:
    # prompt = f"""
    #       You are a pragmatic but quality-focused marketing reviewer. You're given:

    #       Product Description:
    #       {description}

    #       Marketing Plan:
    #       {plan}

    #       Critique Output (feedback identifying areas for improvement):
    #       {critique}

    #       Q1: Does the plan sufficiently address the most important points from the critique?
    #       Q2: Is it reasonable to consider this plan final and usable, even if minor enhancements are possible?

    #       ----
    #       Please answer using the format:
    #       <final>yes</final>  or  <final>no</final>

    #       --- Reasoning ---
    #       Provide a concise and practical explanation. If the plan resolves most key issues, is implementation-ready with minor flaws, and is not just an outline, say "yes". If major gaps remain, say "no".
    #       """

    prompt = f"""
          You are a senior marketing executive responsible for approving final plans before launch. You are expected to uphold high standards but not aim for perfection.

          Product Description:
          {description}

          Marketing Plan:
          {plan}

          Critique Output:
          {critique}

          Q1: Does the marketing plan **explicitly address all major points** from the critique?
          Q2: Is the plan comprehensive, clearly structured, and **ready for implementation with no significant issues**?
          Q3: Would you approve this plan to be sent to stakeholders or clients?

          --- Response Format ---
          <final>yes</final> or <final>no</final>

          --- Reasoning ---
          Provide a clear, structured evaluation:
          - Bullet list of major critique points addressed (or not addressed)
          - Highlight any remaining gaps
          - Explain if the plan is truly implementation-ready or still needs revision

          Approve if the plan addresses the critique points.
          If major gaps remain, say "no". If the plan is not ready for implementation, say "no".
          If the plan resolves gaps and issues by the critique and is implementation-ready, say "yes".
          """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a pragmatic, results-oriented marketing reviewer. "
                    "Your goal is to approve solid plans that meet core goals, not to demand perfection."
                )
            },
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=600
    )
    return response.choices[0].message.content.strip()


In [ ]:
# 🔍 Test Each Agent One by One
initial_plan = writing_agent(product_description)
print("--- Initial Plan ---\n", initial_plan)

critique = critiquing_agent(product_description, initial_plan)
print("\n--- Critique ---\n", critique)

final_decision = decision_agent(product_description, initial_plan, critique)
print("\n--- Decision ---\n", final_decision)

--- Initial Plan ---
 I. Executive Summary: 

The marketing plan for AquaGlow, a smart, eco-friendly water bottle aims to position the product as a health and wellness essential that encourages hydration and supports sustainable living. Our primary target audience will be health-conscious and tech-savvy individuals, looking to make sustainable choices. The plan involves a strategic mix of online and offline marketing, leveraging social media, influencer marketing, partnerships, and events.

II. Market Analysis:

Our market analysis will focus on understanding the size of the health and wellness market, particularly the segment interested in hydration and sustainability. We'll also analyze the competition, market trends, and consumer behavior to develop a robust marketing strategy.

III. Target Audience:

Our primary target audience is health-conscious individuals, fitness enthusiasts, and tech-savvy consumers aged between 18-40. The secondary audience includes professionals and parents

In [ ]:
# 🔁 Agent Loop (up to max_iter iterations)
max_iter = 10
plans = []

# Initial draft
plan = writing_agent(product_description)
print("\n📦 Initial Plan:\n", plan)

for i in range(max_iter):
    plans.append(plan)
    critique = critiquing_agent(product_description, plan)
    decision = decision_agent(product_description, plan, critique)

    print("="*100)
    print(f"\n🔄 Iteration {i+1}")
    print("-"*80)
    print(f"\n Decision: {decision}")
    # print("-"*80)
    # print(f"\n Critique: {critique}")

    if "<final>yes</final>" in decision.lower():
        print("✅ Final plan accepted.")
        break
    elif i == max_iter - 1:
        print("❌ Maximum number of iterations reached. Stopping.")
        break
    else:
        # Revise using critique + previous version
        plan = writing_agent(product_description, critique, plan)

print("\n📦 Final Plan:\n", plan)



📦 Initial Plan:
 I. Executive Summary:
The marketing plan for AquaGlow, an eco-friendly water bottle that syncs with mobile health apps and uses LED light cues to remind users to stay hydrated, will focus on promoting the product's unique features and benefits. Our target market will primarily include health-conscious individuals, fitness enthusiasts, and tech-savvy consumers. 

II. Market Analysis:
This will involve understanding the current market trends, customer needs and preferences, competitor analysis, and potential opportunities. We will identify our target segments and create buyer personas to effectively tailor our marketing strategies.

III. Marketing Goals:
1. Create brand awareness and establish AquaGlow as a premium and innovative water bottle in the market.
2. Drive sales through online and offline channels.
3. Increase customer engagement and loyalty through regular interactions and feedback mechanisms.

IV. Marketing Strategies:
1. Digital Marketing: Utilize SEO, PPC,

# ✅ Conclusion: What You Learned
This notebook showed a hands-on implementation of **Agentic AI** using OpenAI's language models.

- You learned how to structure a multi-agent loop involving writing, critique, and decision-making.
- You saw how orchestrating agents can iteratively refine outputs.

This is a basic example, but the pattern can scale to more complex workflows in real-world applications.
Try modifying the agents or tasks to explore further possibilities of agentic systems!